In [ ]:
import sys
import spotipy
import spotipy.util as util
from spotipy import oauth2
import json
import os
import re

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
scope = 'user-library-read'

username = '7yp23sdg5ptloppevj62zjn6a'

token = util.prompt_for_user_token(username, scope,client_id = "944cef2fde194a8e8d4d5e8db666e020",
                                   client_secret = "52f43621583e4844a4728a5577617f24",
                                   redirect_uri = 'https://example.com/callback/')#, SPOTIPY_REDIRECT_URI)

if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    for item in results['items']:
        track = item['track']
        print (track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print ("Can't get token for", username)

# Obtaining information for a specific user

In [ ]:
username = input("What's the username?")

sp = spotipy.Spotify(auth=token)
playlists = sp.user_playlists(username)
#print(json.dumps(playlists, indent =2))

In [ ]:
print("The playlist name is " + playlists['items'][0]['name'])
print("the playlist id is " + (playlists['items'][0]['uri']).split("playlist:")[1])
print("The total number of songs is " + str(playlists['items'][0]['tracks']['total']))

In [ ]:
playlist_name = []
playlist_id = []
playlist_total_songs = []

for i in playlists['items']:
    playlist_name.append(i['name'])
    playlist_id.append(i['uri'].split("playlist:")[1])
    playlist_total_songs.append(str(i['tracks']['total']))
    
print(len(playlist_name))
print(playlist_name)
print(len(playlist_id))
print(playlist_id)
print(len(playlist_total_songs))
print(playlist_total_songs)

In [ ]:
user_playlists_info = {'Playlist Name' : playlist_name, 'Playlist ID' : playlist_id, 
                       'Playlist_Total_Songs' : playlist_total_songs}

# Obtaining the list of all songs in each playlist

In [ ]:
playlist_info = {}

playlist_name = []
playlist_owner = []
owner_type = []
playlist_followers = []
artist_id = []
artist_name = []
album_name = []
song_name = []
song_id = []
song_popularity = []



for p_id in playlist_id:
    #print(p_id)
    try:
        results = sp.user_playlist(username, p_id)
    except Exception:
        print('Cannot find following playlist id: ' + p_id)
        continue
    
    print(json.dumps(results,indent=4))

    for i in results['tracks']['items']:
        playlist_name.append(results['name'])
        playlist_owner.append(results['owner']['id'])
        owner_type.append(results['owner']['type'])
        playlist_followers.append(results['followers']['total'])
        artist_id.append(i['track']['artists'][0]['id'])
        artist_name.append(i['track']['artists'][0]['name'])
        album_name.append(i['track']['album']['name'])
        song_name.append(i['track']['name'])
        song_id.append(i['track']['id'])
        song_popularity.append(i['track']['popularity'])
        
    
# print(playlist_name)
# print(playlist_owner)
# print(owner_type)
# print(playlist_followers)
# print(artist_id)
# print(artist_name)
# print(album_name)
# print(song_name)
# print(song_id)
# print(song_popularity)
# print(len(song_popularity))

In [ ]:
playlists_info = {'Playlist Name' : playlist_name, 'Playlist Owner' : playlist_owner, 'Owner Type' : owner_type, 
                 'Playlist Followers' : playlist_followers, 'Artist ID' : artist_id, 'Artist Name' : artist_name,
                 'Album Name' : album_name, 'Song Name' : song_name, 'Song ID' : song_id, 'Song Popularity' : song_popularity}

In [ ]:
#print(playlists_info)

In [ ]:
playlists_info_df = pd.DataFrame(playlists_info, columns = ['Playlist Name', 'Playlist Owner', 'Owner Type', 
                 'Playlist Followers', 'Artist ID', 'Artist Name','Album Name', 'Song Name', 'Song ID', 'Song Popularity'])

In [ ]:
playlists_info_df.head()

In [ ]:
playlists_info_df.info()

# Obtaining the features for each song contained in the playlist

In [ ]:
song_id_2 = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []

for s_id in song_id:
    features = sp.audio_features(s_id)
    #print(features)
    #print(json.dumps(features, indent = 4))
    for feature in features:
        song_id_2.append(s_id)
        danceability.append(feature['danceability'])
        energy.append(feature['energy'])
        key.append(feature['key'])
        loudness.append(feature['loudness'])
        mode.append(feature['mode'])
        speechiness.append(feature['speechiness'])
        acousticness.append(feature['acousticness'])
        instrumentalness.append(feature['instrumentalness'])
        liveness.append(feature['liveness'])
        valence.append(feature['valence'])
        tempo.append(feature['tempo'])
        duration_ms.append(feature['duration_ms'])

In [ ]:
songs_info = {'Song ID' : song_id_2,'Danceability' : danceability, 'Energy' : energy, 'Key' : key, 'Loudness' : loudness,
              'Mode' : mode, 'Speechiness' : speechiness, 'Acousticness' : acousticness, 'Instrumentalness' : instrumentalness,
              'Liveness' : liveness, 'Valence' : valence, 'Tempo' : tempo, 'Duration ms' : duration_ms}

In [ ]:
songs_info_df = pd.DataFrame(songs_info, columns = ['Song ID','Danceability', 'Energy', 'Key', 'Loudness',
              'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness','Liveness', 'Valence', 'Tempo', 'Duration ms'])

In [ ]:
songs_info_df.info()

In [ ]:
songs_info_df = pd.merge(songs_info_df,playlists_info_df[['Song ID','Song Popularity']],on='Song ID', how='left')

In [ ]:
songs_info_df.head()

In [ ]:
#Custome color palette
red_blue = ['#19B5FE', '#EF4836']
palette = sns.color_palette(red_blue)
sns.set_palette(palette)
sns.set_style("white")

In [ ]:
pos_tempo = songs_info_df[songs_info_df['Song Popularity'] > 50]['Tempo']
neg_tempo = songs_info_df[songs_info_df['Song Popularity'] < 50]['Tempo']
pos_dance = songs_info_df[songs_info_df['Song Popularity'] > 50]['Danceability']
neg_dance = songs_info_df[songs_info_df['Song Popularity'] < 50]['Danceability']
pos_duration = songs_info_df[songs_info_df['Song Popularity'] > 50]['Duration ms']
neg_duration = songs_info_df[songs_info_df['Song Popularity'] < 50]['Duration ms']
pos_loudness = songs_info_df[songs_info_df['Song Popularity'] > 50]['Loudness']
neg_loudness = songs_info_df[songs_info_df['Song Popularity'] < 50]['Loudness']
pos_speechiness = songs_info_df[songs_info_df['Song Popularity'] > 50]['Speechiness']
neg_speechiness = songs_info_df[songs_info_df['Song Popularity'] < 50]['Speechiness']
pos_valence = songs_info_df[songs_info_df['Song Popularity'] > 50]['Valence']
neg_valence = songs_info_df[songs_info_df['Song Popularity'] < 50]['Valence']
pos_energy = songs_info_df[songs_info_df['Song Popularity'] > 50]['Energy']
neg_energy = songs_info_df[songs_info_df['Song Popularity'] < 50]['Energy']
pos_acousticness = songs_info_df[songs_info_df['Song Popularity'] > 50]['Acousticness']
neg_acousticness = songs_info_df[songs_info_df['Song Popularity'] < 50]['Acousticness']
pos_key = songs_info_df[songs_info_df['Song Popularity'] > 50]['Key']
neg_key = songs_info_df[songs_info_df['Song Popularity'] < 50]['Key']
pos_instrumentalness = songs_info_df[songs_info_df['Song Popularity'] > 50]['Instrumentalness']
neg_instrumentalness = songs_info_df[songs_info_df['Song Popularity'] < 50]['Instrumentalness']

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.title("Song tempo high/low popularity distribution")
pos_tempo.hist(alpha=0.7, bins = 30, label='high popularity')
neg_tempo.hist(alpha=0.7, bins = 30, label='low popularity')
plt.legend(loc = 'upper right')

In [ ]:
fig2 = plt.figure(figsize=(15,15))

# Danceability
ax3 = fig2.add_subplot(331)
ax3.set_xlabel('Danceability')
ax3.set_ylabel('Count')
ax3.set_title('Song Danceability Popularity Distribution')
pos_dance.hist(alpha=0.5, bins=30)
ax4 = fig2.add_subplot(331)
neg_dance.hist(alpha=0.5, bins=30)

# Duration
ax5 = fig2.add_subplot(332)
ax5.set_xlabel('Duration MS')
ax5.set_ylabel('Count')
ax5.set_title('Song Duration Populatiry Distribution')
pos_duration.hist(alpha=0.5, bins=30)
ax6 = fig2.add_subplot(332)
neg_duration.hist(alpha=0.5, bins=30)

# Loudness
ax7 = fig2.add_subplot(333)
ax7.set_xlabel('Loudness')
ax7.set_ylabel('Count')
ax7.set_title('Song Loudness Popularity Distribution')
pos_loudness.hist(alpha=0.5, bins=30)
ax8 = fig2.add_subplot(333)
neg_loudness.hist(alpha=0.5, bins=30)

# Speechiness
ax9 = fig2.add_subplot(334)
ax9.set_xlabel('Speechiness')
ax9.set_ylabel('Count')
ax9.set_title('Song Speechiness Popularity Distribution')
pos_speechiness.hist(alpha=0.5, bins=30)
ax10 = fig2.add_subplot(334)
neg_speechiness.hist(alpha=0.5, bins=30)

# Valence
ax11 = fig2.add_subplot(335)
ax11.set_xlabel('Valence')
ax11.set_ylabel('Count')
ax11.set_title('Song Valence Popularity Distribution')
pos_valence.hist(alpha=0.5, bins=30)
ax12 = fig2.add_subplot(335)
neg_valence.hist(alpha=0.5, bins=30)

# Energy
ax13 = fig2.add_subplot(335)
ax13.set_xlabel('Energy')
ax13.set_ylabel('Count')
ax13.set_title('Song Energy Popularity Distribution')
pos_energy.hist(alpha=0.5, bins=30)
ax14 = fig2.add_subplot(335)
neg_energy.hist(alpha=0.5, bins=30)

# Acousticness
ax15 = fig2.add_subplot(336)
ax15.set_xlabel('Acousticness')
ax15.set_ylabel('Count')
ax15.set_title('Song Acousticness Popularity Distribution')
pos_acousticness.hist(alpha=0.5, bins=30)
ax16 = fig2.add_subplot(336)
neg_acousticness.hist(alpha=0.5, bins=30)

# Key
ax17 = fig2.add_subplot(337)
ax17.set_xlabel('Key')
ax17.set_ylabel('Count')
ax17.set_title('Song Key Popularity Distribution')
pos_key.hist(alpha=0.5, bins=30)
ax18 = fig2.add_subplot(337)
neg_key.hist(alpha=0.5, bins=30)

# Instrumentalness
ax19 = fig2.add_subplot(338)
ax19.set_xlabel('Instrumentalness')
ax19.set_ylabel('Count')
ax19.set_title('Song Instrumentalness Popularity Distribution')
pos_instrumentalness.hist(alpha=0.5, bins=30)
ax20 = fig2.add_subplot(338)
neg_instrumentalness.hist(alpha=0.5, bins=30)


# Using K-Means to create some clusters

In [ ]:
from sklearn.cluster import KMeans

import hypertools as hyp

km = KMeans(n_clusters=4, init='k-means++', n_init=10)

In [ ]:
songs_info_clusters = songs_info_df.drop(['Song ID', 'Song Popularity', 'Key', 'Mode', 'Valence', 'Tempo', 'Duration ms'], axis = 1)
songs_info_clusters.head()

In [ ]:
km.fit(songs_info_clusters)

In [ ]:
x = km.fit_predict(songs_info_clusters)
x

In [ ]:
songs_info_clusters["Cluster"]= x
songs_info_clusters.head()

In [ ]:
target = songs_info_clusters['Cluster']
songs_info_clusters = songs_info_clusters.drop('Cluster', axis=1)
group = target.astype('str')

In [ ]:
songs_info_df["Cluster"]= x
songs_info_df.head()

In [ ]:
songs_info_df_sorted_cluster = songs_info_df.sort_values(['Cluster'])
songs_info_df_sorted_cluster = songs_info_df_sorted_cluster.reset_index(drop=True)
songs_info_df_sorted_cluster

In [ ]:
plt.scatter(songs_info_df_sorted_cluster['Cluster'], songs_info_df_sorted_cluster['Danceability'],
            c=x, s=50, cmap='viridis')
plt.show()

In [ ]:
plt.scatter(songs_info_df_sorted_cluster['Cluster'], songs_info_df_sorted_cluster['Energy'],
            c=x, s=50, cmap='viridis')
plt.show()

In [ ]:
plt.scatter(songs_info_df_sorted_cluster['Cluster'], songs_info_df_sorted_cluster['Loudness'],
            c=x, s=50, cmap='viridis')
plt.show()

In [ ]:
plt.scatter(songs_info_df_sorted_cluster['Cluster'], songs_info_df_sorted_cluster['Speechiness'],
            c=x, s=50, cmap='viridis')
plt.show()

In [ ]:
plt.scatter(songs_info_df_sorted_cluster['Cluster'], songs_info_df_sorted_cluster['Acousticness'],
            c=x, s=50, cmap='viridis')
plt.show()

In [ ]:
plt.scatter(songs_info_df_sorted_cluster['Cluster'], songs_info_df_sorted_cluster['Instrumentalness'],
            c=x, s=50, cmap='viridis')
plt.show()

In [ ]:
plt.scatter(songs_info_df_sorted_cluster['Cluster'], songs_info_df_sorted_cluster['Liveness'],
            c=x, s=50, cmap='viridis')
plt.show()

In [ ]:
hyp.plot(songs_info_clusters, '.', group=group, model='TSNE', ndims=2, legend=[1,2,3,4], 
         title = "2D Representation of the Main Cluster using t-SNE")

In [ ]:
hyp.plot(songs_info_clusters, '.', group=group, model='PCA', ndims=2, legend=[1,2,3,4], 
         title = "2D Representation of the Main Cluster using PCA")

In [ ]:
hyp.plot(songs_info_clusters, '.', group=group, model='TSNE', ndims=3, legend=[1,2,3,4], 
         title = "3D Representation of the Main Cluster using t-SNE")